In [17]:
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np

import dat_analysis
from dat_analysis import useful_functions as U

default_config = dict(
    toImageButtonOptions={'format': 'png',  # one of png, svg, jpeg, webp
                      'scale': 2  # Multiply title/legend/axis/canvas sizes by this factor
})

default_layout = dict(template="plotly_white",
            xaxis=dict(
                mirror=True,
                ticks='outside',
                showline=True,
                linecolor='black',
            ),
            yaxis=dict(
                mirror=True,
                ticks='outside',
                showline=True,
                linecolor='black',
            ))

def get_dat(datnum):
    return dat_analysis.get_dat(datnum, host='qdev-xld', user='Tim', experiment='202208_KondoConductanceDots')



In [20]:
datnums = range(198, 202+1)
dats = [get_dat(num) for num in datnums]

fig_isense, fig_isense_diff, fig_conductance = go.Figure(), go.Figure(), go.Figure()
fig_isense.update_layout(title='Charge Sensor Current',
                        coloraxis_colorbar_title='Current /nA')
fig_isense_diff.update_layout(title='Charge Sensor Current Differentiated',
                        coloraxis_colorbar_title='dI/dVg /nA')
fig_conductance.update_layout(title='Conductance through QD',
                             coloraxis_colorbar_title='Current /nA')

for fig in fig_isense, fig_conductance, fig_isense_diff:
    fig.update_layout(**default_layout,
                     xaxis_title='RC1', 
                     yaxis_title="N1")
    
for dat in dats:
    x = dat.Data.x
    y = dat.Data.y
    
    # i_sense
    data = dat.Data.get_data('standard/i_sense')
    data, x = U.resample_data(data, x=x, max_num_pnts=300, resample_x_only=True)
    fig_isense.add_trace(go.Heatmap(x=x, y=y, z=data, coloraxis='coloraxis'))
    
    # i_sense_diff
    data = np.diff(data)
    x = U.get_matching_x(x, data)
    fig_isense_diff.add_trace(go.Heatmap(x=x, y=y, z=data, coloraxis='coloraxis'))
    
    # conductance
    data = dat.Data.get_data('current_2d')
    x = dat.Data.x
    data, x = U.resample_data(data, x=x, max_num_pnts=300, resample_x_only=True)
    fig_conductance.add_trace(go.Heatmap(x=x, y=y, z=data, coloraxis='coloraxis'))
    
for fig in fig_isense, fig_isense_diff, fig_conductance:
    fig.show(config=default_config)